In [8]:
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from data.extraction import get_sp500
import yaml
from tqdm import notebook

In [9]:
cookies = {
    '_clck': '1grxf1v|2|fbl|0|1229',
    '_ga': 'GA1.1.1352746695.1684077774',
    '__cf_bm': 'Q37GL_25LtWRK64er_9MPxFGQmg9jXQGM3HfWE3UG.8-1684081502-0-AfFHvUv5q7IKuRlq2jTdHjGWwpcceyn0ovAE7dpD8OFIVWYxGoip4QzOlkz0zagOuUpOxij7ARHUdBb2ZWOPAsBvFtJTvoq8FSfvJC0YTISf',
    'otpid': '64610b9039b54fbcc1c5e001',
    'token': 'Bearer%20eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJlbWFpbCI6ImRla292aWMuZGFyaW9AZ21haWwuY29tIiwibmFtZSI6IkRhcmlvIERla292acSHIiwidGllciI6InN1cHBvcnRlciIsImNyZWF0ZWRBdCI6IjIwMjMtMDUtMTRUMTU6MjY6MTkuOTYyWiIsImlhdCI6MTY4NDA4MTU3MSwiZXhwIjoxNzE1NjM4NDk3fQ.W_KvQE48Nr29TfzLaePRvJJ0jYn6pfFXhFDHFFRDALM',
    '_ga_65KH44GT56': 'GS1.1.1684077774.1.1.1684081606.0.0.0',
    '_clsk': 'ibfps|1684082196149|60|1|s.clarity.ms/collect',
}

headers = {
    'authority': 'roic.ai',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9,hr;q=0.8',
    'cache-control': 'max-age=0',
    # 'cookie': '_clck=1grxf1v|2|fbl|0|1229; _ga=GA1.1.1352746695.1684077774; __cf_bm=Q37GL_25LtWRK64er_9MPxFGQmg9jXQGM3HfWE3UG.8-1684081502-0-AfFHvUv5q7IKuRlq2jTdHjGWwpcceyn0ovAE7dpD8OFIVWYxGoip4QzOlkz0zagOuUpOxij7ARHUdBb2ZWOPAsBvFtJTvoq8FSfvJC0YTISf; otpid=64610b9039b54fbcc1c5e001; token=Bearer%20eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJlbWFpbCI6ImRla292aWMuZGFyaW9AZ21haWwuY29tIiwibmFtZSI6IkRhcmlvIERla292acSHIiwidGllciI6InN1cHBvcnRlciIsImNyZWF0ZWRBdCI6IjIwMjMtMDUtMTRUMTU6MjY6MTkuOTYyWiIsImlhdCI6MTY4NDA4MTU3MSwiZXhwIjoxNzE1NjM4NDk3fQ.W_KvQE48Nr29TfzLaePRvJJ0jYn6pfFXhFDHFFRDALM; _ga_65KH44GT56=GS1.1.1684077774.1.1.1684081606.0.0.0; _clsk=ibfps|1684082196149|60|1|s.clarity.ms/collect',
    'referer': 'https://roic.ai/company/GOOG?utm_source=main',
    'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}

params = {
    'fs': 'quarterly',
}

In [10]:
with open("config.yaml") as config_fh:
    try:
        config = yaml.safe_load(config_fh)["API_CONNECTION"]
    except yaml.YAMLError as e:
        print(f"An error occured when reading YAML file: {e}")

symbols = get_sp500(config["SYMBOL_URL"])
output_path = "dataset/raw_data/roic_data/"

for i, ticker in enumerate(notebook.tqdm(symbols.loc[:, "Symbol"])):
    if ticker == "FI":
        ticker = "FISV"
    soup = BeautifulSoup(requests.get(f'https://roic.ai/financials/{ticker}', params=params, cookies=cookies, headers=headers).content, "html.parser")
    data = json.loads(soup.select_one("#__NEXT_DATA__").text)
    #data = json.loads(soup.select_one("#__NEXT_DATA__").text)

    # umcomment this to print all data:
    # print(json.dumps(data["props"]["pageProps"]["data"]["data"], indent=4))

    df_bsq = pd.DataFrame(data["props"]["pageProps"]["data"]["data"]["bsq"])
    df_cfq = pd.DataFrame(data["props"]["pageProps"]["data"]["data"]["cfq"])
    df_isq = pd.DataFrame(data["props"]["pageProps"]["data"]["data"]["isq"])

    df_bsq["symbol"] = df_bsq["symbol"].replace("FISV", "FI")
    df_cfq["symbol"] = df_cfq["symbol"].replace("FISV", "FI")
    df_isq["symbol"] = df_isq["symbol"].replace("FISV", "FI")

    if i != 0:
        df_bsq.to_csv(f"{output_path}bsq.csv", mode='a', header=False, index=False)
        df_cfq.to_csv(f"{output_path}cfq.csv", mode='a', header=False, index=False)
        df_isq.to_csv(f"{output_path}isq.csv", mode='a', header=False, index=False)
    else:
        df_bsq.to_csv(f"{output_path}bsq.csv", header=True, index=False)
        df_cfq.to_csv(f"{output_path}cfq.csv", header=True, index=False)
        df_isq.to_csv(f"{output_path}isq.csv", header=True, index=False)

  0%|          | 0/503 [00:00<?, ?it/s]